In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
movie_df = pd.read_csv('/content/drive/MyDrive/ml-latest/movies.csv')

In [ ]:
movie_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [ ]:
rating_df = pd.read_csv('/content/drive/MyDrive/ml-latest/ratings.csv')

In [ ]:
rating_df

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [ ]:
# 가장 많이 평점을 입력한 top5 유저id 찾기
rating_top = rating_df.groupby('userId').count()
rating_top = rating_top.sort_values('rating', ascending=False)
rating_top.head(5)

,movieId,rating,timestamp
userId,,,
123100,23715,23715,23715
117490,9279,9279,9279
134596,8381,8381,8381
212343,7884,7884,7884
242683,7515,7515,7515


In [ ]:
# 혼자 천상계에 있는 123100 유저 제외하고 2~4위를 3명으로 선정했다고 가정. 내가 분석할 유저는 212343

In [ ]:
# 212343이 rating을 입력한 영화 살펴보기
rating_by_212343_df = rating_df[rating_df['userId'] == 212343]
rating_by_212343_df = rating_by_212343_df.sort_values('movieId')
rating_by_212343_df

,userId,movieId,rating,timestamp
20797356,212343,1,3.5,1436135504
20797357,212343,2,1.5,1436135389
20797358,212343,3,3.0,1436135323
20797359,212343,4,1.0,1436135292
20797360,212343,5,2.5,1436135295
...,...,...,...,...
20805235,212343,189885,2.5,1529431119
20805236,212343,191157,3.5,1531805008
20805237,212343,192659,2.0,1534733035
20805238,212343,192917,2.5,1535607481


In [ ]:
# 반대로 212343이 rating을 안입력한 영화 살펴보기... 많다.
not_rated_by_212343_df = rating_df[rating_df['userId'] != 212343]
not_rated_by_212343_df = not_rated_by_212343_df.sort_values('movieId')
not_rated_by_212343_df

,userId,movieId,rating,timestamp
19441972,198468,1,3.0,867342869
25580791,261290,1,3.0,834963473
15771197,161112,1,4.0,1246722364
24807660,253513,1,3.0,867050708
19711135,201107,1,4.0,851610733
...,...,...,...,...
17318700,176871,193878,2.0,1537875080
7939059,81710,193880,2.0,1537886232
3236396,33330,193882,2.0,1537890512
22170940,226715,193886,4.0,1537916177


In [ ]:
# 212343이 뭔 영화를 봤는지 살펴보자.
rating_by_212343_df = rating_by_212343_df.reset_index()
rating_by_212343_df.drop('index', axis=1, inplace=True)

In [ ]:
rating_by_212343_df = rating_by_212343_df.set_index('movieId')

In [ ]:
movie_df = movie_df.set_index('movieId')

In [ ]:
# 이런 영화들을 봤구먼... 근데 영화 장르가 보기 불편하게 되어있다. 전체 장르 중 해당하는 영화의 장르면 1, 아니면 0으로 labeling해보자.
movie_seened_by_212343_df = movie_df.loc[rating_by_212343_df.index]
movie_seened_by_212343_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
189885,The Guilty (2018),Thriller
191157,Relative Evil (2001),Comedy|Drama|Thriller
192659,Liar's Edge (1992),Thriller


In [ ]:
# 전체 영화 데이터프레임의 장르부터 다 바꿔주자.
movie_df['genres'] = movie_df['genres'].str.split('|')
movie_df

,title,genres
movieId,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
5,Father of the Bride Part II (1995),[Comedy]
...,...,...
193876,The Great Glinka (1946),[(no genres listed)]
193878,Les tribulations d'une caissière (2011),[Comedy]
193880,Her Name Was Mumu (2016),[Drama]


In [ ]:
# 전체 장르가 몇개인지 확인
genres_list = []
for row in movie_df.itertuples():
  for i in row.genres:
    genres_list.append(i)
genres_list = sorted(list(set(genres_list)))
print(genres_list)

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
# 장르별로 칼럼을 일단 만들어주자.
movie_df[genres_list] = 0

In [ ]:
# 반복문을 돌면서, 해당 영화가 해당 장르이면 1로 labeling하자.
for row in movie_df.itertuples():
    for i in row.genres:
        movie_df.loc[row.Index, i] = 1

In [ ]:
# IMAX가 장르는 아니니 지우고, 원래 있던 장르 칼럼도 지워주자. 그리고 (no genres listed)는 이름 좀 바꿔주자.
movie_df.drop(['genres', 'IMAX'], axis=1, inplace = True)
movie_df.rename(columns = {'(no genres listed)':'No genres'})

,title,No genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193876,The Great Glinka (1946),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
193878,Les tribulations d'une caissière (2011),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
193880,Her Name Was Mumu (2016),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 다시 212343이 무슨 영화를 봤는지 살펴보자.
movie_seened_by_212343_df = movie_df.loc[rating_by_212343_df.index]
movie_seened_by_212343_df

,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189885,The Guilty (2018),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
191157,Relative Evil (2001),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
192659,Liar's Edge (1992),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# 212343이 위에 있는 영화를 보고 rating을 준거다. 한 눈에 보기 위해 합쳐보자.
seen_by_212343_data = pd.concat([rating_by_212343_df, movie_seened_by_212343_df], axis=1)
seen_by_212343_data

,userId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,212343,3.5,1436135504,Toy Story (1995),0,0,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,212343,1.5,1436135389,Jumanji (1995),0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,212343,3.0,1436135323,Grumpier Old Men (1995),0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,212343,1.0,1436135292,Waiting to Exhale (1995),0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,212343,2.5,1436135295,Father of the Bride Part II (1995),0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189885,212343,2.5,1529431119,The Guilty (2018),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
191157,212343,3.5,1531805008,Relative Evil (2001),0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
192659,212343,2.0,1534733035,Liar's Edge (1992),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# 212343이 주로 몇 점을 rating 했을까? rating으로 위 데이터를 groupby 해보자. 본 영화의 합이 맞는지도 확인해보자.
seen_by_212343_data_groupby_rating = seen_by_212343_data.groupby('rating').count()
seen_by_212343_data_groupby_rating['userId'].sum()

7884

In [ ]:
# 맞다. 그러면 몇개 봤는지 count로 깔끔히 볼수있게 하자.
seen_by_212343_data_count = seen_by_212343_data_groupby_rating[['userId']]
seen_by_212343_data_count.rename(columns = {'userId':'count'}, inplace = True)
seen_by_212343_data_count

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,count
rating,
0.5,432
1.0,778
1.5,510
2.0,1158
2.5,1049
3.0,1529
3.5,1554
4.0,560
4.5,259


##1

In [ ]:
# 시각화해보자.
import plotly.graph_objects as go

fig = px.line(seen_by_212343_data_count, x=seen_by_212343_data_count.index, y="count", title = 'user_212343 movie_rating_count', text = 'count')
fig.update_xaxes(dtick=0.5)
fig.show()

# 212343의 rating 평점을 보면, 주로 3.5점을 많이 주고 4점부터 5점은 별로 없다.

In [ ]:
seen_by_212343_data_genres_sum = seen_by_212343_data.groupby('rating').sum()
seen_by_212343_data_genres_sum

,userId,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
rating,,,,,,,,,,,,,,,,,,,,,
0.5,91732176,624696348008,4,94,45,7,13,157,60,16,...,26,1,45,12,28,57,53,104,12,5
1.0,165202854,1122169366144,1,145,88,13,27,283,87,8,...,61,3,110,14,48,161,74,186,29,16
1.5,108294930,736968068245,1,72,40,5,19,138,47,5,...,30,1,100,12,42,93,43,142,13,3
2.0,245893194,1669537538576,2,180,80,9,17,390,134,11,...,52,10,154,25,82,252,86,295,55,30
2.5,222747807,1516436935208,4,161,74,13,15,307,155,11,...,43,4,197,11,83,163,62,368,33,25
3.0,324672447,2201748028124,6,229,120,8,30,438,260,15,...,58,20,187,17,149,235,93,527,51,36
3.5,329981022,2238439568864,3,213,106,18,25,410,312,29,...,49,14,148,25,133,187,75,503,64,47
4.0,118912080,804524086282,0,74,56,18,25,151,102,15,...,28,22,28,16,44,67,27,127,60,16
4.5,54996837,371789773666,0,33,24,3,8,64,61,5,...,12,10,15,8,28,42,15,69,21,14


In [ ]:
# userId와 timestamp는 여기선 떨궈주자.
seen_by_212343_data_genres_sum.drop(['userId', 'timestamp'], axis=1, inplace = True)

In [ ]:
# 별점별, 장르별을 한눈에 보고싶으면 축을 한번 바꿔줘보자.
seen_by_212343_data_genres_sum.transpose()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
(no genres listed),4,1,1,2,4,6,3,0,0,1
Action,94,145,72,180,161,229,213,74,33,2
Adventure,45,88,40,80,74,120,106,56,24,2
Animation,7,13,5,9,13,8,18,18,3,2
Children,13,27,19,17,15,30,25,25,8,3
Comedy,157,283,138,390,307,438,410,151,64,12
Crime,60,87,47,134,155,260,312,102,61,18
Documentary,16,8,5,11,11,15,29,15,5,0
Drama,215,369,277,624,514,901,1000,395,170,38
Fantasy,26,61,30,52,43,58,49,28,12,3


##2

In [ ]:
# 시각화해보자.
fig = px.bar(seen_by_212343_data_genres_sum.transpose(), title = 'user_212343 movie_genres_by_rating')
fig.show()

# 드라마, 코메디, 스릴러 등등 순으로 많이 봤다.

In [ ]:
# 근데 얘가 이걸 많이 보고 싶어서 많이 본걸까? 영화 데이터에 해당 장르 영화가 많아서 얘를 많이 본 것 아닐까? 이를 확인하기위해 전체 데이터 대비 위 데이터로 비율 계산을 해보자.
# 전체 데이터 장르 개수 확인, 영화 당 장르에 중복이 있기 때문에 실제 영화 개수인 58000개보다 많다.
movie_df_genres_sum = pd.DataFrame(movie_df.sum()[1:])
movie_df_genres_sum

,0
(no genres listed),4266
Action,7130
Adventure,4067
Animation,2663
Children,2749
Comedy,15956
Crime,5105
Documentary,5118
Drama,24144
Fantasy,2637


In [ ]:
movie_df_genres_sum.rename(columns = {0:'all_genre_count'},inplace=True)

In [ ]:
seen_by_212343_data_genres_sum_f = pd.DataFrame(seen_by_212343_data_genres_sum.sum())

In [ ]:
seen_by_212343_data_genres_sum_f.rename(columns = {0:'212343_genre_count'},inplace=True)

In [ ]:
# 데이터셋 전체 영화 장르 수 대비 212343이 본 장르의 수는 다음과 같다.
seen_by_212343_movie_genre_partition = pd.concat([movie_df_genres_sum, seen_by_212343_data_genres_sum_f], axis=1)
seen_by_212343_movie_genre_partition

,all_genre_count,212343_genre_count
(no genres listed),4266,22
Action,7130,1203
Adventure,4067,635
Animation,2663,96
Children,2749,182
Comedy,15956,2350
Crime,5105,1236
Documentary,5118,115
Drama,24144,4503
Fantasy,2637,362


In [ ]:
# 비율도 계산해보자.
seen_by_212343_movie_genre_partition = seen_by_212343_movie_genre_partition.astype({'all_genre_count':int})
seen_by_212343_movie_genre_partition['212343_partition'] = round((seen_by_212343_movie_genre_partition['212343_genre_count']/seen_by_212343_movie_genre_partition['all_genre_count'])*100, 2)

In [ ]:
# 비율 추가 완료.
seen_by_212343_movie_genre_partition = seen_by_212343_movie_genre_partition.sort_values('212343_partition', ascending=False)
seen_by_212343_movie_genre_partition

,all_genre_count,212343_genre_count,212343_partition
Thriller,8216,2332,28.38
Crime,5105,1236,24.21
Film-Noir,364,88,24.18
Mystery,2773,641,23.12
War,1820,347,19.07
Drama,24144,4503,18.65
Horror,5555,985,17.73
Romance,7412,1263,17.04
Action,7130,1203,16.87
Adventure,4067,635,15.61


## 3

In [ ]:
# 위 데이터를 시각화해보자.
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.3, 0.7])
# fig.add_trace(go.Line(x=partition.index, y=partition['212343_partition'], name="partition", text=partition['212343_partition'], mode="lines+markers+text", textposition= "middle right"), row=1, col=1)
fig.add_trace(go.Bar(x=seen_by_212343_movie_genre_partition.index, y=seen_by_212343_movie_genre_partition['212343_partition'], name="partition", text=seen_by_212343_movie_genre_partition['212343_partition'], textposition="outside"), row=1, col=1)
fig.add_trace(go.Bar(x=seen_by_212343_movie_genre_partition.index, y=seen_by_212343_movie_genre_partition['all_genre_count'], name="all_movie_genres", text=seen_by_212343_movie_genre_partition['all_genre_count'], textposition="outside"), row=2, col=1)
fig.add_trace(go.Bar(x=seen_by_212343_movie_genre_partition.index, y=seen_by_212343_movie_genre_partition['212343_genre_count'], name="212343_movie_genres", text=seen_by_212343_movie_genre_partition['212343_genre_count'], textposition="outside"), row=2, col=1)

fig.update_layout(
    title=dict(
        text = "all vs 212343 movie_count and partition",
        x=0.5, y=0.9,
        )
    )

fig.show()

# 역시, 비율로 따지니 212343이 좋아하는 장르 순이 바뀌었다. 

In [ ]:
# 이제 이 장르 대로 212343이 안 본 영화를 정리해보자.
not_rated_by_212343_df_groupby_genres = not_rated_by_212343_df.groupby('movieId').mean()
not_rated_by_212343_df_groupby_genres

,userId,rating,timestamp
movieId,,,
1,141938.209587,3.886655,1.128090e+09
2,142392.716859,3.246647,1.105949e+09
3,140367.259304,3.173993,9.692634e+08
4,140503.956158,2.875167,9.404215e+08
5,141249.728366,3.077328,9.970528e+08
...,...,...,...
193876,103565.000000,3.000000,1.537875e+09
193878,176871.000000,2.000000,1.537875e+09
193880,81710.000000,2.000000,1.537886e+09


In [ ]:
not_seen_by_212343_movie_df = movie_df.loc[not_rated_by_212343_df_groupby_genres.index]
not_seen_by_212343_movie_df

,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193876,The Great Glinka (1946),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
193878,Les tribulations d'une caissière (2011),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
193880,Her Name Was Mumu (2016),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 안 본 영화 중 212343이 선호하는 장르의 영화 선별(비율대로 많이 본 순 5개)
not_seen_by_212343_movie_df_top5 = not_seen_by_212343_movie_df[(not_seen_by_212343_movie_df['Thriller']==1)|(not_seen_by_212343_movie_df['Crime']==1)|(not_seen_by_212343_movie_df['Mystery']==1)|(not_seen_by_212343_movie_df['Film-Noir']==1)|(not_seen_by_212343_movie_df['War']==1)]

In [ ]:
not_seen_by_212343_movie_df_top5 = not_seen_by_212343_movie_df_top5.reset_index()
not_seen_by_212343_movie_df_top5

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,6,Heat (1995),0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,10,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,16,Casino (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,20,Money Train (1995),0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
4,21,Get Shorty (1995),0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12980,193843,Raazi (2018),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12981,193845,Dywizjon 303 (2018),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12982,193851,U: July 22 (2018),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12983,193861,Cold Harvest (1998),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [ ]:
# 212343이 rating 안 준 영화 -> 데이터 많은게 재밌으니까 rating 조건 걸기 및 sort는 안하겠음
# not_rated_by_212343_df = not_rated_by_212343_df[not_rated_by_212343_df['rating'] >= 4]
# not_rated_by_212343_df = not_rated_by_212343_df.sort_values('rating', ascending=False)
not_rated_by_212343_df = not_rated_by_212343_df.reset_index(drop=True)

In [ ]:
not_rated_by_212343_df

,userId,movieId,rating,timestamp
0,198468,1,3.0,867342869
1,261290,1,3.0,834963473
2,161112,1,4.0,1246722364
3,253513,1,3.0,867050708
4,201107,1,4.0,851610733
...,...,...,...,...
27745555,176871,193878,2.0,1537875080
27745556,81710,193880,2.0,1537886232
27745557,33330,193882,2.0,1537890512
27745558,226715,193886,4.0,1537916177


In [ ]:
# 위 데이터에서 212343의 선호 장르에 해당하는 영화 id의 행만 가져오기
final_data = not_rated_by_212343_df[not_rated_by_212343_df['movieId'].isin(not_seen_by_212343_movie_df_top5['movieId'])]
final_data

,userId,movieId,rating,timestamp
129655,183750,6,3.0,857915212
129656,277081,6,5.0,1097520094
129657,242725,6,3.5,1275235314
129658,231078,6,5.0,1114780711
129659,38333,6,5.0,1251843253
...,...,...,...,...
27745537,81924,193845,2.0,1537919848
27745540,106822,193851,3.5,1537805917
27745545,1686,193861,3.0,1537816652
27745558,226715,193886,4.0,1537916177


In [ ]:
final_data = final_data.reset_index(drop=True)

In [ ]:
# 근데 겁나 많다... 랜덤으로 100만개만 뽑은거 하나 가져가고, 하나는 전체로 가져가보자.
import random
final_data_random_index = random.sample(list(final_data.index), 1000000)
final_data_random = final_data.loc[final_data_random_index]
final_data_random

,userId,movieId,rating,timestamp
5396921,199490,2455,3.0,974759178
357590,278741,63,3.0,845984922
5932240,93731,2890,4.0,943835681
3888759,131404,1347,0.5,1273463851
7126697,138512,4226,3.5,1142872880
...,...,...,...,...
4076167,64476,1411,4.0,1239417080
10928365,103020,105504,3.5,1418308031
2709806,214581,839,4.0,1253835223
4472014,67848,1674,5.0,938943686


In [ ]:
final_data_random = final_data_random.reset_index(drop=True)

In [ ]:
rating_by_212343_df = rating_by_212343_df.reset_index()

In [ ]:
# 요기에 212343이 봤던 영화들의 데이터를 살포시 concat해주자.
final_data_random = pd.concat([final_data_random, rating_by_212343_df], ignore_index = True)
final_data_random.tail(10)

,userId,movieId,rating,timestamp
1007874,212343,187489,1.0,1527412723
1007875,212343,188679,2.0,1530503507
1007876,212343,189071,2.5,1530490522
1007877,212343,189141,2.0,1532920805
1007878,212343,189485,3.0,1534736643
1007879,212343,189885,2.5,1529431119
1007880,212343,191157,3.5,1531805008
1007881,212343,192659,2.0,1534733035
1007882,212343,192917,2.5,1535607481
1007883,212343,193551,2.0,1537080716


In [ ]:
# 마찬가지로 원래 final data에도 212343의 영화데이터를 concat해주자.
final_data = pd.concat([final_data, rating_by_212343_df], ignore_index = True)
final_data

,userId,movieId,rating,timestamp
0,183750,6,3.0,857915212
1,277081,6,5.0,1097520094
2,242725,6,3.5,1275235314
3,231078,6,5.0,1114780711
4,38333,6,5.0,1251843253
...,...,...,...,...
11315851,212343,189885,2.5,1529431119
11315852,212343,191157,3.5,1531805008
11315853,212343,192659,2.0,1534733035
11315854,212343,192917,2.5,1535607481


In [ ]:
# 파일로 만들어서 저장하자.
final_data_random.to_csv('212343_final_data_random.csv')

In [ ]:
final_data.to_csv('212343_final_data.csv')